## setting up the base directory and initial configurations
    ### To Do
        * preliminary analysis
        * preprocessing
            * identify categorical variables
            * missing value imputation
        * Feature Selection
            * run rf with selected features
            * use GA to select features
            * encode GA urself
            

In [37]:
folder_name = "/springleaf_marketing_response/"
base_dir = "/Users/gulshan/kaggle/"
data_dir = base_dir+"/data/"+folder_name
output_dir = base_dir+"/output/"+folder_name

In [38]:
%matplotlib inline

In [39]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [5]:
train_data_orig=pd.read_csv(data_dir+"/train.csv")
test_data_orig=pd.read_csv(data_dir+"/test.csv")
print train_data_orig.shape,test_data_orig.shape

(145231, 1934) (145232, 1933)


## Preliminary Analysis Pipeline
    * dtype of columns
    * missing value check
    * distribution plots

In [6]:
train_copy=train_data_orig.copy()
test_copy=test_data_orig.copy()

In [27]:

def check_for_non_numeric(value):
    if type(value)==str:
        return("str",value)
    else:
        return("NA","NA")
    
def check_for_non_numeric_column(pandas_series):
    uniq_values=pandas_series.unique()
    for value in uniq_values:
        if type(value)==str:
            return True
            break
       

        
            

def get_missing_value_count(input_df):
    """
    return counter dict where keys are column names and values are number of missing values
    {
    'VAR_0669': 0,
    'VAR_0460': 918,
    }
    """
    missing_value_counter={}
    colnames=input_df.columns.values
    for names in colnames:
        tmp_df=pd.isnull(input_df[names])
        count=len(tmp_df[tmp_df==True])
        missing_value_counter[names]=count
    return missing_value_counter


In [8]:
def know_ur_data(df):
    data_descriptor={}
    df_describe={}
    colnames=df.columns.values
    
    for name in colnames[:5]:
        tmp=list(df[name].apply(np.isreal).unique())
        if False in tmp:
            data_descriptor['str_flag']="str"
        else:
            data_descriptor['str_flag']="real"
        
        data_descriptor['mean']=pd.DataFrame(df[name]).mean(0).values.tolist()
        data_descriptor['mode']=pd.DataFrame(df[name]).mode(0).values.tolist()
        data_descriptor['max']=pd.DataFrame(df[name]).max(0).values.tolist()
        data_descriptor['min']=pd.DataFrame(df[name]).min(0).values.tolist()
        df_describe[name]=data_descriptor
        
        
    return df_describe

In [9]:
train_copy.head(1)

,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,...,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0


In [11]:
test_describe=know_ur_data(test_copy)
train_describe=know_ur_data(train_copy)

In [12]:
train_describe

{'ID': {'max': ['S'],
  'mean': [],
  'min': ['B'],
  'mode': [['B']],
  'str_flag': 'str'},
 'VAR_0001': {'max': ['S'],
  'mean': [],
  'min': ['B'],
  'mode': [['B']],
  'str_flag': 'str'},
 'VAR_0002': {'max': ['S'],
  'mean': [],
  'min': ['B'],
  'mode': [['B']],
  'str_flag': 'str'},
 'VAR_0003': {'max': ['S'],
  'mean': [],
  'min': ['B'],
  'mode': [['B']],
  'str_flag': 'str'},
 'VAR_0004': {'max': ['S'],
  'mean': [],
  'min': ['B'],
  'mode': [['B']],
  'str_flag': 'str'},
 'VAR_0005': {'max': ['S'],
  'mean': [],
  'min': ['B'],
  'mode': [['B']],
  'str_flag': 'str'}}

In [13]:
pd.DataFrame(train_copy['VAR_0004']).mode(0).values

array([[2000]])

In [25]:
# replace with the mode
def missing_value_imputation(df,colnames,df_stats,stats_param):
    """
    in place imputation for missing values
    """
    for key in colnames:
        print key
        filler=df_stats[key][stats_param][0]
        df[key]=df[key].fillna(value=filler)

    

In [28]:
colnames=train_copy.columns.values    
str_colnames=[x for x in colnames if check_for_non_numeric_column(train_copy[x])]
n_cat=[len(set(train_copy[x])) for x in str_colnames]

In [54]:
colnames_test=test_copy.columns.values    
str_colnames_test=[x for x in colnames_test if check_for_non_numeric_column(test_copy[x])]
n_cat_test=[len(set(train_copy[x])) for x in str_colnames_test]

#SVD

In [34]:
svd_data=train_copy[[x for x in colnames if x not in str_colnames]]
svd_data.shape

(145231, 1896)

In [36]:
svd_data_matrix=svd_data.as_matrix()

In [ ]:
float_arr = np.vstack(svd_data_matrix).astype(np.float)

In [55]:
svd_data_test=test_copy[[x for x in colnames_test if x not in str_colnames_test]]
print svd_data_test.shape
svd_data_matrix_test=svd_data_test.as_matrix()
float_arr_test = np.vstack(svd_data_matrix_test).astype(np.float)

(145232, 1895)


In [51]:
with open(output_dir+'float_arr.pickle', 'wb') as handle:
    pickle.dump(float_arr, handle)
u,s, vt = svds(float_arr, k = 500)
 
with open(output_dir+'svd_u.pickle', 'wb') as handle:
    pickle.dump(u, handle)
with open(output_dir+'svd_s.pickle', 'wb') as handle:
    pickle.dump(s, handle)
with open(output_dir+'svd_vt.pickle', 'wb') as handle:
    pickle.dump(vt, handle)

In [52]:
s_diag_matrix = np.zeros((s.shape[0], s.shape[0]))

for i in range(s.shape[0]):
    s_diag_matrix[i,i] = s[i]

In [53]:
X_lr = np.dot(np.dot(u, s_diag_matrix), vt)

In [96]:
s_diag_matrix_t.shape,u_t.shape,vt_t.shape

((500, 500), (145232, 500), (500, 1895))

In [56]:
with open(output_dir+'float_arr_test.pickle', 'wb') as handle:
    pickle.dump(float_arr_test, handle)
u_t,s_t, vt_t = svds(float_arr_test, k = 500)
 
with open(output_dir+'svd_u_test.pickle', 'wb') as handle:
    pickle.dump(u_t, handle)
with open(output_dir+'svd_s_test.pickle', 'wb') as handle:
    pickle.dump(s_t, handle)
with open(output_dir+'svd_vt_test.pickle', 'wb') as handle:
    pickle.dump(vt_t, handle)

s_diag_matrix_t = np.zeros((s_t.shape[0], s_t.shape[0]))



NameError: name 'u_T' is not defined

In [57]:
for i in range(s_t.shape[0]):
    s_diag_matrix[i,i] = s_t[i]
X_lr_t = np.dot(np.dot(u_t, s_diag_matrix_t), vt_t.transpose())

In [64]:
X_lr_t.shape,X_lr.shape,train_copy.shape,test_copy.shape

((145232, 1895), (145231, 1896), (145231, 1934), (145232, 1933))

In [70]:
a=pd.DataFrame(X_lr_t)
a1=a.drop(a.columns[[0, 1894]], axis=1)
a1.head(1)

,1,2,3,4,5,6,7,8,9,10,...,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [80]:

target_t=train_copy['target']
X_lr_dataframe=pd.DataFrame(X_lr)
train_t=X_lr_dataframe.drop(X_lr_dataframe.columns[[0, 1895]], axis=1)

In [81]:

# target_t=test_copy['target']
X_lr_t_dataframe=pd.DataFrame(X_lr_t)
test_t=X_lr_t_dataframe.drop(X_lr_t_dataframe.columns[[0]], axis=1)

In [87]:
train_t.shape,test_t.shape,target_t.shape


((145231, 1894), (145232, 1894), (145231,))

In [171]:
# missing_value_columns

In [90]:
missing_value_dict=get_missing_value_count(train_copy)
missing_value_columns=[x for x in missing_value_dict.keys() if missing_value_dict[x]>0]
missing_value_str_columns=list(set(missing_value_columns).intersection(set(str_colnames)))

In [110]:
missing_value_imputation(train_copy,list(set(missing_value_columns)-set(missing_value_str_columns)),train_describe,'mode')

VAR_0460


ValueError: invalid fill value with a <type 'numpy.ndarray'>

In [316]:
# missing_value_imputation(train_copy,missing_value_str_columns,"NA")

In [327]:
for key in missing_value_str_columns:
    print key,len(set(train_copy[key]))
#     if len(set(train_copy[key]))<10:
#         print set(train_copy[key])

In [318]:
# convert VAR_0217 to month
#VAR_0200 is location names 
#VAR_0404 is designation VAR_0493
#VAR_0204 time stamp VAR_0075:more varied

# set(train_copy['VAR_0342'])

In [344]:
train_data=train_copy[list(set(colnames)-set(str_colnames))]


In [357]:
# col=train_data.columns.values
# for key in col:
#     if "26APR12:00:00:00" in list(train_data[key]):
#         print key 
#VAR_0073 VAR_0156 VAR_0157 VAR_0158 VAR_0159

In [358]:
target=train_data['target']
train=train_data.drop(['ID','target'], 1)

### Test Data

In [346]:
colnames_test=test_copy.columns.values
str_colnames_test=[x for x in colnames_test if get_type(test_copy[x])==str]

In [285]:
missing_value_dict=get_missing_value_column_dict(test_copy)
missing_value_columns=[x for x in missing_value_dict.keys() if missing_value_dict[x]>0]
missing_value_str_columns=list(set(missing_value_columns).intersection(set(str_colnames_test)))

In [286]:
missing_value_imputation(test_copy,list(set(missing_value_columns)-set(missing_value_str_columns)),0)

In [359]:
test_data=test_copy[list(set(colnames_test)-set(str_colnames_test))]
test=test_data.drop('ID',1)

##Preprocessing Pipeline
    * Imputation of missing values
    * Encoding categorical features
    * Normalization

In [ ]:
class preprocess:
    def one_hot_encoding(input_array):
        
        
    

In [88]:

# Create the random forest object which will include all the parameters
# for the fit
rf = RandomForestClassifier(n_estimators = 100,n_jobs=-1, verbose =1)
%time rf.fit(train_t, target_t)

[Parallel(n_jobs=-1)]: Done   1 out of 100 | elapsed:   11.3s remaining: 18.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.3min finished


CPU times: user 15min 30s, sys: 9.13 s, total: 15min 39s
Wall time: 5min 23s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=None, verbose=1,
            warm_start=False)

In [89]:
submission=pd.DataFrame()
ID=test_copy['ID']
output = rf.predict(test_t)
# output=[0]*len(ID)
submission['ID']=ID
submission['target']=output
submission.to_csv(output_dir+'submission_svd.csv',index = False)

[Parallel(n_jobs=4)]: Done   1 out of  21 | elapsed:    0.0s remaining:    0.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.9s finished


In [91]:
submission.head()

,ID,target
0,1,0
1,3,0
2,6,0
3,9,0
4,10,0
